In [1]:
# pip install unidecode
# pip install selenium
# pip install bs4

# Sử dụng selenium lấy dữ liệu các bài báo từ công cụ Google Search

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import os
from time import sleep
from selenium.webdriver.common.by import By

def initDriver():

    WINDOW_SIZE = "1000,2000"
    chrome_options = Options()
    #chrome_options.add_argument("--headless")
    chrome_options.add_argument("--window-size=%s" % WINDOW_SIZE)
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('disable-infobars')
    chrome_options.add_argument('--disable-gpu') if os.name == 'nt' else None  # Windows workaround
    chrome_options.add_argument("--verbose")
    chrome_options.add_argument("--no-default-browser-check")
    chrome_options.add_argument("--ignore-ssl-errors")
    chrome_options.add_argument("--allow-running-insecure-content")
    chrome_options.add_argument("--disable-web-security")
    chrome_options.add_argument("--disable-feature=IsolateOrigins,site-per-process")
    chrome_options.add_argument("--no-first-run")
    chrome_options.add_argument("--disable-notifications")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-translate")
    chrome_options.add_argument("--ignore-certificate-error-spki-list")
    chrome_options.add_argument("--ignore-certificate-errors")
    chrome_options.add_argument("--disable-blink-features=AutomationControllered")
    chrome_options.add_experimental_option('useAutomationExtension', False)
    prefs = {"profile.default_content_setting_values.notifications": 2}
    chrome_options.add_experimental_option("prefs", prefs)
    chrome_options.add_argument("--start-maximized")  # open Browser in maximized mode
    chrome_options.add_argument("--disable-dev-shm-usage")  # overcome limited resource problems
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option("prefs", {"profile.managed_default_content_settings.images": 2})
    chrome_options.add_argument('disable-infobars')

    driver = webdriver.Chrome(options=chrome_options)
    return driver


In [3]:

driver = initDriver()


## Sửa từ khóa tìm kiếm

In [4]:
from selenium.webdriver.common.keys import Keys
keyword = "vụ tấn công ở đắk lắk"
# keyword = "tổng bí thư trung quốc đến thăm việt nam"

driver.get("https://www.google.com/search?sca_esv=599503376&sxsrf=ACQVn0-YkRlT1UNLhRnMzK7gcVba-gwsLQ:1705596004849&tbm=nws&q="+keyword)
# inputbox = driver.find_element(By.XPATH,"//textarea[@id='APjFqb']")
# inputbox.send_keys()
# inputbox.send_keys(Keys.ENTER)
# for i in range(10):
#     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")



In [5]:

# def get_info(item):
#     link = item.find_element(By.XPATH,"//a[@jsname='UWckNb']").get_attribute("href")
#     name = item.find_element(By.XPATH,"//h3[@class='LC20lb MBeuO DKV0Md']").text
#     time = item.find_element(By.XPATH,"//span[@class='LEwnzc Sqrs4e']").text
#     des= item.find_element(By.XPATH,"//div[@class='VwiC3b yXK7lf lVm3ye r025kc hJNv6b Hdw6tb']/span[2]").text


In [6]:
class GoogleResult:
    def __init__(self):
        self.name = None
        self.link = None
        self.description = None
        self.content = None
        self.time = None
    def show(self):
        print("link : ",self.link)
        print("time : ",self.time)
        print("name : ",self.name)
        print("content : ",self.content)
        print("description : ",self.description)
    def returnjson(self):
        return {"name":self.name,
                "time":self.time,
                "link":self.link,
                "content":self.content,
                "description":self.description
                }
        
def get_info(item):
    obj = GoogleResult()
    
# def get_info(item):
#     link = item.find_element(By.XPATH,"//a[@jsname='UWckNb']").get_attribute("href")
#     name = item.find_element(By.XPATH,"//h3[@class='LC20lb MBeuO DKV0Md']").text
#     time = item.find_element(By.XPATH,"//span[@class='LEwnzc Sqrs4e']").text
#     des= item.find_element(By.XPATH,"//div[@class='VwiC3b yXK7lf lVm3ye r025kc hJNv6b Hdw6tb']/span[2]").text

    try:
        obj = GoogleResult()
        obj.link =  item.find('a')['href']
        obj.name = item.find('div',{'class','n0jPhd ynAwRc MBeuO nDgy9d'}).text
        obj.time = item.find('div',{'class','OSrXXb rbYSKb LfVVr'}).text
        obj.description = item.find('div',{'class','GI74Re nDgy9d'}).text
        
    except Exception as e :
        print(e)
    
    
    return obj


In [7]:
from bs4 import BeautifulSoup
html_source = driver.page_source

soup = BeautifulSoup(html_source, 'html.parser')

In [8]:
from bs4 import BeautifulSoup
import time 

data = []
for _ in range(20):
    html_source = driver.page_source

    soup = BeautifulSoup(html_source, 'html.parser')

    listall = soup.findAll('div', {'class','SoaBEf'})    

    for item in listall:
        temp = get_info(item)
        data.append(temp.returnjson())
        
    url = soup.find_all('td',{'class','d6cvqb BBwThe'})[1].find('a')['href']
    driver.get("https://www.google.com/"+url)
    

In [9]:
from unidecode import unidecode
import json 

fileoutput = unidecode(keyword).replace(" ","_")

print(fileoutput)

with open(fileoutput+".json", "w") as f:
    f.write(json.dumps(data, indent=4))

vu_tan_cong_o_dak_lak


In [10]:
# driver.quit()

In [11]:
len(data)

200

## Lấy nội dung của một trang web cụ thể

In [12]:
item = {'name': 'Ngày mai 16-1, xét xử 100 bị cáo tấn công trụ sở UBND 2 xã ở Đắk Lắk',
  'time': '4 ngày trước',
  'link': 'https://tuoitre.vn/ngay-mai-16-1-xet-xu-100-bi-cao-tan-cong-tru-so-ubnd-2-xa-o-dak-lak-20240115113427307.htm',
  'description': 'Ngày mai (16-1), TAND tỉnh Đắk Lắk sẽ đưa ra xét xử lưu động 100 bị cáo \ntrong vụ tấn công trụ sở UBND 2 xã ở Đắk Lắk rạng sáng 11-6-2023.'}

item['name']

'Ngày mai 16-1, xét xử 100 bị cáo tấn công trụ sở UBND 2 xã ở Đắk Lắk'

In [13]:
url = item['link']
driver = initDriver()
driver.get(url)
html_source = driver.page_source

soup = BeautifulSoup(html_source, 'html.parser')

target_tag = soup.find('h1', text=item['name'])    

# Nếu thẻ tồn tại, tìm thẻ cha của nó
if target_tag:
    parent_tag = target_tag.find_parent()
    print("Đã tìm thấy Thẻ cha")
else:
    print("Không tìm thấy thẻ")

content = soup.find("div", {"class":parent_tag.attrs['class'][0]}).get_text() 


Đã tìm thấy Thẻ cha


<ipython-input-13-a537a2b22781>:8: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  target_tag = soup.find('h1', text=item['name'])


In [14]:
parent_tag.attrs['class'][0]


'detail__cmain'

In [15]:
soup.find("div", {"class":parent_tag.attrs['class'][0]}).get_text()

'\n\n\nPháp luật\n\n\n\n                    15/01/2024 12:35 GMT+7\n                \n\n\nNgày mai 16-1, xét xử 100 bị cáo tấn công trụ sở UBND 2 xã ở Đắk Lắk\n\n\n\n\n\n\n\n\nTRUNG TÂN\n\n\n\n\n\n\n\n\n\nNghe đọc bài 2:34  1x  0.5x  0.75x  1x  1.25x  1.5x    Nữ miền Bắc   Nữ miền BắcNam miền BắcNam miền NamNữ miền Nam\n\n\n\n\n\n\n\n\n            Ngày mai (16-1), TAND tỉnh Đắk Lắk sẽ đưa ra xét xử lưu động 100 bị cáo trong vụ tấn công trụ sở UBND 2 xã ở Đắk Lắk rạng sáng 11-6-2023.\n        \n\n\n\n\n\nCơ quan công an bắt một trong những đối tượng tham gia vụ khủng bố - Ảnh: T.DŨNGNgày 15-1, tại cuộc họp báo định kỳ, bà H\'Yim Kđoh - phó chủ tịch UBND tỉnh Đắk Lắk - thông tin một số nội dung trước phiên xét xử 100 bị cáo tham gia tấn công trụ sở UBND 2 xã ở Đắk Lắk.Vụ tấn công trụ sở UBND 2 xã có tính chất đặc biệt nghiêm trọngTheo bà H\'Yim, quá trình điều tra cho thấy rạng sáng 11-6-2023, gần 100 người bịt mặt mang theo nhiều vũ khí, chia làm 2 tốp, tấn công trụ sở UBND hai xã, làm 

In [16]:
def get_content(item):
    url = item['link']
    driver.get(url)
    html_source = driver.page_source
    soup = BeautifulSoup(html_source, 'html.parser')

    target_tag = soup.find('h1', string=item['name'])    

    # Nếu thẻ tồn tại, tìm thẻ cha của nó
    if target_tag:
        parent_tag = target_tag.find_parent()
        print("Đã tìm thấy Thẻ cha")
        content = soup.find("div", {"class":parent_tag.attrs['class'][0]}).get_text() 
        return content
    else:
        print("Không tìm thấy thẻ")
        return ""

    


In [18]:
from bs4 import BeautifulSoup
import time 

keyword = "vụ tấn công ở đắk lắk"
# keyword = "tổng bí thư trung quốc đến thăm việt nam"

driver.get("https://www.google.com/search?sca_esv=599503376&sxsrf=ACQVn0-YkRlT1UNLhRnMzK7gcVba-gwsLQ:1705596004849&tbm=nws&q="+keyword)


data = []
for _ in range(20):
    html_source = driver.page_source

    soup = BeautifulSoup(html_source, 'html.parser')

    listall = soup.findAll('div', {'class','SoaBEf'})    

    for item in listall:
        temp = get_info(item)
        temp.content = get_content(temp.returnjson())
        print("\n_________________________________\n")
        temp.show()
        data.append(temp.returnjson())
        
    url = soup.find_all('td',{'class','d6cvqb BBwThe'})[1].find('a')['href']
    driver.get("https://www.google.com/"+url)

<ipython-input-16-04280b6bbbfe>:7: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  target_tag = soup.find('h1', text=item['name'])


Đã tìm thấy Thẻ cha

_________________________________

link :  https://tuoitre.vn/ngay-mai-16-1-xet-xu-100-bi-cao-tan-cong-tru-so-ubnd-2-xa-o-dak-lak-20240115113427307.htm
time :  4 ngày trước
name :  Ngày mai 16-1, xét xử 100 bị cáo tấn công trụ sở UBND 2 xã ở Đắk Lắk
content :  


Pháp luật



                    15/01/2024 12:35 GMT+7
                


Ngày mai 16-1, xét xử 100 bị cáo tấn công trụ sở UBND 2 xã ở Đắk Lắk








TRUNG TÂN









Nghe đọc bài 2:34  1x  0.5x  0.75x  1x  1.25x  1.5x    Nữ miền Bắc   Nữ miền BắcNam miền BắcNam miền NamNữ miền Nam








            Ngày mai (16-1), TAND tỉnh Đắk Lắk sẽ đưa ra xét xử lưu động 100 bị cáo trong vụ tấn công trụ sở UBND 2 xã ở Đắk Lắk rạng sáng 11-6-2023.
        





Cơ quan công an bắt một trong những đối tượng tham gia vụ khủng bố - Ảnh: T.DŨNGNgày 15-1, tại cuộc họp báo định kỳ, bà H'Yim Kđoh - phó chủ tịch UBND tỉnh Đắk Lắk - thông tin một số nội dung trước phiên xét xử 100 bị cáo tham gia tấn công trụ sở UBND 2 xã

In [1]:
fileoutput = unidecode(keyword).replace(" ","_")

print(fileoutput)

with open("content_"+fileoutput+".json", "w") as f:
    f.write(json.dumps(data, indent=4))

NameError: name 'unidecode' is not defined